In [1]:
import tensorflow as tf


2023-03-27 03:53:22.970941: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-27 03:53:23.023180: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-27 03:53:23.846953: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
BASE_PATH = '/home/ec2-user/isolated-sign-language-recognition/sample-data'

In [4]:
index = pd.read_csv('index.csv')
# sample_data = pd.read_csv(BASE_PATH)

In [5]:
index = index.drop(columns='path')
index.head()

,participant_id,sequence_id,sign
0,26734,1000035562,blow
1,28656,1000106739,wait
2,16069,100015657,cloud
3,25571,1000210073,bird
4,62590,1000240708,owie


In [6]:
class_labels = index.sign.unique()
class_labels

array(['blow', 'wait', 'cloud', 'bird', 'owie', 'duck', 'minemy', 'lips',
       'flower', 'time', 'vacuum', 'apple', 'puzzle', 'mitten', 'there',
       'dry', 'shirt', 'owl', 'yellow', 'not', 'zipper', 'clean',
       'closet', 'quiet', 'have', 'brother', 'clown', 'cheek', 'cute',
       'store', 'shoe', 'wet', 'see', 'empty', 'fall', 'balloon',
       'frenchfries', 'finger', 'same', 'cry', 'hungry', 'orange', 'milk',
       'go', 'drawer', 'TV', 'another', 'giraffe', 'wake', 'bee', 'bad',
       'can', 'say', 'callonphone', 'finish', 'old', 'backyard', 'sick',
       'look', 'that', 'black', 'yourself', 'open', 'alligator', 'moon',
       'find', 'pizza', 'shhh', 'fast', 'jacket', 'scissors', 'now',
       'man', 'sticky', 'jump', 'sleep', 'sun', 'first', 'grass', 'uncle',
       'fish', 'cowboy', 'snow', 'dryer', 'green', 'bug', 'nap', 'feet',
       'yucky', 'morning', 'sad', 'face', 'penny', 'gift', 'night',
       'hair', 'who', 'think', 'brown', 'mad', 'bed', 'drink', 'stay',


In [7]:
X_train, X_test, y_train, y_test = train_test_split(index[['participant_id','sequence_id']], index['sign'], 
                                                    test_size=0.2, stratify=index['sign'])

train = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
test = pd.concat([X_test, y_test], axis=1).reset_index(drop=True)


In [18]:
y_train

87352     stuck
94320       man
7651     finish
49169       car
25025       red
          ...  
79255    person
33716      doll
42465      bath
16401      make
22438     after
Name: sign, Length: 75581, dtype: object

In [20]:
import glob

def get_train_sample(train_df):
        
    for index in train_df.index:
        metadata = train_df.loc[index]
        files = glob.glob(f"{BASE_PATH}/participant_id={metadata['participant_id']}/sequence_id={metadata['sequence_id']}/*.csv")
        df = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)
        
        df = df.sort_values('frame').values.astype(np.float32)
        
        yield df, metadata['sign']
                
#         # yield df.sort_values('frame').to_numpy()
#         # yield (df.reshape(-1), 0.0)
#         # yield (df.reshape(-1), metadata['sign'])        
    
        
# def get_train_sample(train_df):
        
#     for index in train_df.index:
#         metadata = train_df.loc[index]
#         files = f"{BASE_PATH}/participant_id={metadata['participant_id']}/sequence_id={metadata['sequence_id']}/*.csv"
        
#         yield tf.convert_to_tensor(files,dtype=tf.string), tf.convert_to_tensor(metadata['sign'],dtype=tf.string)        
    

In [21]:
tmp_df = index[(index['participant_id']==16069) & (index['sequence_id']==1004211348)]
# tmp_df = index[(index['participant_id']==16069) & (index['sequence_id']==1005492440)]



In [22]:
# tf.convert_to_tensor(next(get_train_sample(tmp_df)))
next(get_train_sample(tmp_df))

(array([[106.        ,   0.41268227,   0.42548835, ...,   0.        ,
           0.        ,   0.        ]], dtype=float32),
 'bug')

In [32]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [24]:
# dataset = tf.data.Dataset.from_generator(get_train_sample, args=[tmp_df],
#                                          output_types=tf.string)



In [33]:
train_dataset = tf.data.Dataset.from_generator(
    generator=lambda: get_train_sample(tmp_df),
    output_signature=(
        tf.TensorSpec(shape=(1, 1087), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.string)
    )
)

In [34]:
for features, label in train_dataset:
    print("Features:", features)
    print("Label:", label)

Features: tf.Tensor(
[[106.           0.41268227   0.42548835 ...   0.           0.
    0.        ]], shape=(1, 1087), dtype=float32)
Label: tf.Tensor(b'bug', shape=(), dtype=string)


2023-03-27 04:17:39.166385: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


NameError: name 'train_df' is not defined